In [57]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import json
import urllib
from tqdm import tqdm
import pickle


from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.options import Options as ChromeOptions

In [8]:
data = {"request":{"filters":{"tags":{"slugs":["novosti"],"comparison_type":"and"}},"limit":12,"offset":36},"request_id":83345806,"application_id":19}
data

accept: application/json, text/plain, */*
accept-encoding: gzip, deflate, br
accept-language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7
content-length: 130
content-type: application/json;charset=UTF-8
cookie: _ym_uid=1665222042863863336; _ym_d=1665222042; adtech_uid=afa4c7bb-afe2-4a1d-a306-8d4f9953d911%3Arabota.ru; top100_id=t1.7512535.2053263024.1665222041970; _ym_isad=1; _ym_visorc=w; last_visit=1665200781974%3A%3A1665222381974; t3_sid_7512535=s1.1102837621.1665222041973.1665222911121.1.50.50.1; qrator_ssid=1665222929.528.r8HuyRr53OmUoDKf-mcpatvl714d6785lsceiu9huge5pnf6v
origin: https://prosto.rabota.ru
referer: https://prosto.rabota.ru/category/news
sec-ch-ua: "Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105"
sec-ch-ua-mobile: ?0
sec-ch-ua-platform: "Linux"
sec-fetch-dest: empty
sec-fetch-mode: cors
sec-fetch-site: same-origin
user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36


In [ ]:
req = requests.post('Request URL: https://prosto.rabota.ru/api-web/v4/blog/posts.json',
                   headers = myheaders,
                   data = '{"clientId":"androidapp"}',
#                    proxies = {'all' : 'socks5://{}'.format('45.94.47.108:8152')}
                  )

## Get links to all posts

In [9]:
url = "https://prosto.rabota.ru/category/news"
headers = {"User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}

In [155]:
chrome_options = ChromeOptions()
#chrome_options.add_argument("--headless")
driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [12]:
driver.get(url)

In [59]:
for i in tqdm(range(100)):
    driver.find_element_by_tag_name('footer').find_element_by_tag_name('button').click()
    time.sleep(1)

100%|█████████████████████████████████████████| 100/100 [11:29<00:00,  6.89s/it]


In [60]:
x = driver.find_elements_by_class_name('post-snippet-content__title')

In [61]:
len(x)

1379

In [63]:
links = []
for el in tqdm(x):
    links.append(el.find_element_by_tag_name('a').get_attribute('href'))

100%|███████████████████████████████████████| 1379/1379 [02:56<00:00,  7.83it/s]


In [64]:
links[:10]

['https://prosto.rabota.ru/post/putin-osvobodil-ot-mobilizacii-studentov-negosudarstvennyh-vuzov',
 'https://prosto.rabota.ru/post/rossiyane-schitayut-chto-uchitelya-dolzhny-poluchat-70-120-tysyach-v-mesyac',
 'https://prosto.rabota.ru/post/vrachi-i-kosmonavty-samye-slozhnye-professii-po-mneniyu-rossiyan',
 'https://prosto.rabota.ru/post/sotrudniki-it-kompanij-isklyuchennyh-iz-reestra-lishatsya-otsrochki-ot-mobilizacii',
 'https://prosto.rabota.ru/post/rabota-ru-preduprezhdaet-o-moshennikah',
 'https://prosto.rabota.ru/post/rabotnikam-malogo-biznesa-dadut-kratkovremennye-otsrochki',
 'https://prosto.rabota.ru/post/sotrudniki-tin-koff-podali-v-sud-na-bank',
 'https://prosto.rabota.ru/post/nakopleniya-sokratilis-u-kazhdogo-pyatogo-rossiyanina-i-kazhdogo-chetvertogo-muzhchiny',
 'https://prosto.rabota.ru/post/tret-rossijskih-kompanij-ishet-bron-dlya-sotrudnikov',
 'https://prosto.rabota.ru/post/apple-relocirovala-bol-shinstvo-rossijskih-sotrudnikov-v-kirgiziyu']

In [66]:
with open('post_likns.pk', 'wb') as f:
    pickle.dump(links, f)

## Parse posts text

In [67]:
with open('post_likns.pk', 'rb') as f:
    links = pickle.load(f)

In [204]:
len(links)

1379

In [206]:
res = []
for link in tqdm(links):
    driver.get(link)
    time.sleep(0.5)
    vacancy = driver.find_elements_by_class_name('r-vacancy-preview-card__title')
    vacancy_names = []
    for el in vacancy:
        vacancy_names.append(el.text)
    vacancy_names = np.unique(vacancy_names).tolist()
    
    data = requests.get(link)
    soup = BeautifulSoup(data.text)
    text = ' '.join([el.text for el in soup.find_all('div', {'class': 'post'}) 
                     if ' Фото: iStock' not in el.text and 'Все\xa0вакансии на\xa0Работе.ру' not in el.text])
    
    res.append({'url': link, 
                'vacancies': vacancy_names,
                'text': text})

100%|█████████████████████████████████████| 1379/1379 [1:17:32<00:00,  3.37s/it]


In [207]:
df = pd.DataFrame(res)
df.to_pickle('news_vacancy.pkl')
df.head()

,url,vacancies,text
0,https://prosto.rabota.ru/post/putin-osvobodil-...,[Водитель-курьер на личном автомобиле (м. Праж...,"Президент РФ Владимир Путин подписал указ, ра..."
1,https://prosto.rabota.ru/post/rossiyane-schita...,[Водитель-курьер на личном автомобиле (м. Праж...,Сервис Работа.ру узнал у наших соотечественни...
2,https://prosto.rabota.ru/post/vrachi-i-kosmona...,[Водитель-курьер на личном автомобиле (м. Праж...,Сервис Работа.ру опросил наших соотечественни...
3,https://prosto.rabota.ru/post/sotrudniki-it-ko...,[Водитель-курьер на личном автомобиле (м. Праж...,"Глава Минцифры РФ Максут Шадаев напомнил, что..."
4,https://prosto.rabota.ru/post/rabota-ru-predup...,[],Сервис Работа.ру получает сообщения о злоумыш...


In [208]:
df.shape

(1379, 3)

In [202]:
df.iloc[-1]['url']

'https://prosto.rabota.ru/post/mintrud-napomnil-chto-mobilizovannyh-nel-zya-uvolit'

In [220]:
with open('news_vacancy.pickle', 'wb') as f:
    pickle.dump(df, f)

In [209]:
pd.read_pickle('news_vacancy.pkl').sample(5)

,url,vacancies,text
927,https://prosto.rabota.ru/post/bolee-poloviny-r...,[],ВЦИОМ опубликовал опрос об отношении граждан ...
1176,https://prosto.rabota.ru/post/zarplatu-v-2021-...,[],В 2021 году индексация зарплаты будет обязате...
865,https://prosto.rabota.ru/post/vse-pensii-v-str...,"[Водитель категории B, Водитель категории C, Н...",Начиная с сегодняшнего дня все пенсионные и н...
559,https://prosto.rabota.ru/post/polovina-rossiya...,"[Водитель на служебном грузовом автомобиле, Во...","Исследование сервиса Работа.ру показало, что ..."
923,https://prosto.rabota.ru/post/novogodnie-kanik...,[],Россиян ждут десятидневные выходные в новый г...


In [217]:
df.iloc[923]['url']

'https://prosto.rabota.ru/post/novogodnie-kanikuly-prodlyatsya-s-31-dekabrya-po-9-yanvarya'

In [214]:
df['vacancies'].apply(lambda x: len(x)).value_counts()

0     475
9     401
7     359
8      60
16     22
6      17
3      14
2      10
1       8
5       5
4       3
15      2
13      1
14      1
10      1
Name: vacancies, dtype: int64